In [1]:
!git clone https://github.com/LinyingLyu/ChronoGPT.git
%cd ChronoGPT

Cloning into 'ChronoGPT'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 65 (delta 39), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (65/65), 30.19 KiB | 7.55 MiB/s, done.
Resolving deltas: 100% (39/39), done.
/content/ChronoGPT


# Load ChronoGPT-instruct

In [2]:
import gc
import torch
import tiktoken
from huggingface_hub import hf_hub_download
from ChronoGPT_instruct import *
import warnings
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = tiktoken.get_encoding("gpt2")

# ----- Build model ONCE using any instruct repo's config -----
base_repo = "manelalab/chrono-gpt-instruct-v1-19991231"

config_path = hf_hub_download(repo_id=base_repo, filename="config.pt")
config = torch.load(config_path, map_location="cpu")

model = ChronoGPT(**config).to(device).half()
model.eval()

if torch.cuda.is_available():
    print("After init:")
    print("Allocated:", round(torch.cuda.memory_allocated() / 1e9, 3), "GB")
    print("Reserved: ", round(torch.cuda.memory_reserved() / 1e9, 3), "GB")

config.pt:   0%|          | 0.00/924 [00:00<?, ?B/s]

After init:
Allocated: 4.645 GB
Reserved:  9.561 GB


In [3]:
def load_year_weights(repo_id: str):
    """
    Load weights from a chrono-gpt-instruct repo *into the existing model*.
    We only touch CPU RAM for the checkpoint; GPU memory stays roughly constant.
    """
    global model

    # Download checkpoint
    bin_path = hf_hub_download(repo_id=repo_id, filename="pytorch_model.bin")

    # Load on CPU to avoid temporary huge GPU allocations
    state_dict = torch.load(bin_path, map_location="cpu")

    with torch.no_grad():
        model.load_state_dict(state_dict)

    del state_dict
    gc.collect()

    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print(f"Loaded weights from {repo_id}")
        print("Allocated:", round(torch.cuda.memory_allocated() / 1e9, 3), "GB")
        print("Reserved: ", round(torch.cuda.memory_reserved() / 1e9, 3), "GB")

# Instruction Following

In [4]:
# 1999 model
load_year_weights("manelalab/chrono-gpt-instruct-v1-19991231")

user_input = "Who is the U.S. president in 2004?"
response = extract_response(
    user_input,
    model=model,
    tokenizer=tokenizer,
    device=device,
    max_tokens=128,
    temperature=0.0
)

print(f"You: {user_input}")
print(f"\nModel (1999): {response}\n")

pytorch_model.bin:   0%|          | 0.00/7.43G [00:00<?, ?B/s]

Loaded weights from manelalab/chrono-gpt-instruct-v1-19991231
Allocated: 4.645 GB
Reserved:  5.228 GB
You: Who is the U.S. president in 2004?

Model (1999): President John F. Kennedy


President John F. Kennedy



In [5]:
# 2014 model on the *same* model object
load_year_weights("manelalab/chrono-gpt-instruct-v1-20141231")

user_input = "Who is the U.S. president in 2004?"
response = extract_response(
    user_input,
    model=model,
    tokenizer=tokenizer,
    device=device,
    max_tokens=128,
    temperature=0.0
)

print(f"You: {user_input}")
print(f"\nModel (2014): {response}\n")

pytorch_model.bin:   0%|          | 0.00/7.43G [00:00<?, ?B/s]

Loaded weights from manelalab/chrono-gpt-instruct-v1-20141231
Allocated: 4.653 GB
Reserved:  5.228 GB
You: Who is the U.S. president in 2004?

Model (2014): The U.S. president in 2004 is George W. Bush.



In [6]:
# 2024 model on the *same* model object
load_year_weights("manelalab/chrono-gpt-instruct-v1-20241231")

user_input = "Who is the U.S. president in 2004?"
response = extract_response(
    user_input,
    model=model,
    tokenizer=tokenizer,
    device=device,
    max_tokens=128,
    temperature=0.0
)

print(f"You: {user_input}")
print(f"\nModel (2024): {response}\n")

pytorch_model.bin:   0%|          | 0.00/7.43G [00:00<?, ?B/s]

Loaded weights from manelalab/chrono-gpt-instruct-v1-20241231
Allocated: 4.653 GB
Reserved:  5.228 GB
You: Who is the U.S. president in 2004?

Model (2024): The U.S. president in 2004 is George W. Bush.

